In [1]:
import h5py
from pathlib import Path
import numpy as np
import math

from utils import plot_imgs
from jointrecog.settings import DATA_PATH

In [2]:
def conv(im):
    return np.flip(np.moveaxis(im, 0, -1), -1)
def crop_hand(img, kp_2D):
    # Find the outer most keypoints to define the tight bounding box
    x_min, x_max = math.ceil(kp_2D[:,0].min()), math.floor(kp_2D[:,0].max())
    y_min, y_max = math.ceil(kp_2D[:,1].min()), math.floor(kp_2D[:,1].max())

    if x_max == img.shape[1]:
        x_max = img.shape[1]-1
    if y_max == img.shape[0]:
        y_max = img.shape[0]-1
    # Joint coordinate maxes
    left_max, right_max = x_min, x_max
    up_max, bottom_max = y_min, y_max
    # Add a buffer around the extracted bounding box
    buff = 8
    if x_min - buff >= 0:
        x_min -= buff
    if x_max + buff < img.shape[1]:
        x_max += buff
    if y_min - buff >= 0:
        y_min -= buff
    if y_max + buff < img.shape[0]:
        y_max += buff
    # Expand rectangle to square by elongating the shorter side of the rectangle
    y_diff = y_max - y_min
    x_diff = x_max - x_min
    if y_diff > x_diff:
        len_to_board_x = img.shape[1] - x_max
        # Check if square would extend beyond image. Extend on other side instead
        if x_min + y_diff < img.shape[1]:
            x_max = x_min + y_diff
        else:
            # Can't extend only on one side, extend on both sides
            x_min = x_max - (y_diff - (len_to_board_x - 1))
            x_max += (len_to_board_x - 1)
    else:
        len_to_board_y = img.shape[0] - y_max
        if y_min + x_diff < img.shape[0]:
            y_max = y_min + x_diff
        else:
            # Can't extend only on one side, extend on both sides
            y_min = y_max - (x_diff - (len_to_board_y - 1))
            y_max += (len_to_board_y - 1)

    # Extract the crop
    img_crop = img[y_min:y_max, x_min:x_max, :]

    assert(x_min >= 0 and x_max < img.shape[1] and y_min >=0 and
        y_max < img.shape[0])
    assert(y_min<y_max and x_min<x_max)

    # Translate the coordinates accordingly
    kp_2D_crop = kp_2D - np.array([[x_min, y_min]])
    # Some coordinates are slighty outside the image border. Fix
    if kp_2D_crop[:,0].max() > img_crop.shape[1]:
        idx = kp_2D_crop[:,0].argmax()
        kp_2D_crop[idx, 0] = img_crop.shape[1]
    if kp_2D_crop[:,1].max() > img_crop.shape[0]:
        idx = kp_2D_crop[:,1].argmax()
        kp_2D_crop[idx, 1] = img_crop.shape[0]

    return img_crop, kp_2D_crop

In [3]:
train = h5py.File(Path(DATA_PATH, 'training.h5'), 'r')['train']

In [4]:
cropped_kp = []
for img, kp in zip(train['img'], train['kp_2D']):
    i, k = crop_hand(conv(img), kp)
    k = k / np.array(i.shape[:2])[::-1]
    cropped_kp.append(k)
cropped_kp = np.stack(cropped_kp)

In [5]:
m = np.mean(cropped_kp, axis=0)
m

array([[ 0.40865918,  0.45789533],
       [ 0.43802365,  0.3417364 ],
       [ 0.43249291,  0.36539183],
       [ 0.42411622,  0.36192796],
       [ 0.41629521,  0.39991149],
       [ 0.43381974,  0.43200111],
       [ 0.43315204,  0.39809707],
       [ 0.43469335,  0.36059724],
       [ 0.42784982,  0.34791589],
       [ 0.423427  ,  0.52973961],
       [ 0.42465896,  0.49016315],
       [ 0.42810474,  0.44159206],
       [ 0.42400355,  0.41166805],
       [ 0.41649457,  0.58897491],
       [ 0.41647365,  0.55974717],
       [ 0.4194568 ,  0.51760699],
       [ 0.41887264,  0.47361361],
       [ 0.4134851 ,  0.64843264],
       [ 0.41413161,  0.62756074],
       [ 0.41657499,  0.60253332],
       [ 0.41212397,  0.54946456]])